# Assignment 10 - Group 7
## Part B 

#### Connect to MYSQL Server

In [ ]:
import pymysql as myDB
conn = myDB.connect('localhost', 'root', 'root') 
cursor = conn.cursor()

sql = ' SHOW DATABASES; '
cursor.execute(sql)
print("The current databases are: \n")
for row in cursor:
    print (row)

### i. Create a database called MYFLTDATA in mysql. If it already exists, delete it.

In [ ]:
# Drop myfltdata database if it exists. 
sql = ' DROP DATABASE IF EXISTS myfltdata; ' 
cursor.execute(sql)

sql = ' CREATE DATABASE myfltdata; ' 
cursor.execute(sql)

sql = ' SHOW DATABASES; '
cursor.execute(sql)

# Confirm myfltdata database was created.
print("The current databases are: \n")
for row in cursor:
    print (row)

In [ ]:
sql = ' USE myfltdata; ' 
cursor.execute(sql)

cursor.close()

### ii. Save flights and weather as SQL tables in the MYFLTDATA database

#### Save flights and weather data as pandas dataframes. 

In [ ]:
import pandas as pd

# Create flights dataframe. 
fl = pd.read_csv('flights.csv')
fl.columns = ['flightId', 'year', 'month', 'day', 'depTime', 'schedDep', 'depDelay', 'arrTime', 'schedArr', 'arrDelay', 'carrier', 'flight', 'tailNum', 'origin', 'dest', 'airTime', 'distance', 'hour', 'minute', 'timeHour']

# Create weather dataframe. 
w = pd.read_csv('weather.csv')
w.columns = ['weatherId', 'origin', 'year', 'month', 'day', 'hour', 'temp', 'dewP', 'humid', 'windDir', 'windSpeed', 'windGust', 'precip', 'pressure', 'visib', 'timeHour']

View dataframes

In [ ]:
fl.head()

In [ ]:
w.head()

#### Remove NaN Values from Dataframes
In order for these data to be properly inserted into a SQL relation, the nan values must be removed. 

In [ ]:
fl.isnull().sum()

In [ ]:
fl = fl[pd.notnull(fl['depTime'])]
fl = fl[pd.notnull(fl['arrTime'])]
fl = fl[pd.notnull(fl['arrDelay'])]
fl.isnull().sum()

In [ ]:
w.isnull().sum()

In [ ]:
w = w[pd.notnull(w['pressure'])]
w = w[pd.notnull(w['windDir'])]
w.isnull().sum()

#### Save dataframes as new csv. 
The new, clean pandas dataframes are ready to be saved as a csv to be loaded into the SQL relation.

In [ ]:
fl.to_csv('flightsnew.csv', index=False, quoting=1)
w.to_csv('weathernew.csv', index=False, quoting=1)

In [ ]:
conn = myDB.connect(host='localhost', user='root', password='root', db='myfltdata', local_infile=True)
cursor = conn.cursor()

#### Connect to myfltdata database and create flights and weather tables. 

In [ ]:
sql = '''DROP TABLE IF EXISTS flight'''
cursor.execute(sql)

In [ ]:
# Create flights table
sql = '''
        CREATE TABLE flights ( flightId INT(6),
        year INT(4),
        month INT(2),
        day INT(2),
        depTime INT(4),
        schedDep INT(4),
        depDelay INT(4),
        arrTime INT(3),
        schedArr INT(4),
        arrDelay INT(4),
        carrier CHAR(4),
        flight CHAR(4),
        tailNum CHAR(8),
        origin CHAR(3),
        dest CHAR(3),
        airTime INT(3),
        distance INT(6),
        hour INT(2),
        minute INT(2),
        timeHour VARCHAR(20),
        PRIMARY KEY (flightId)
        );
    '''
cursor.execute(sql)

In [ ]:
sql = '''DROP TABLE IF EXISTS weather'''
cursor.execute(sql)

In [ ]:
# Create weather table
sql = '''
        CREATE TABLE weather ( weatherId INT(6),
        origin CHAR(3),
        year INT(4),
        month INT(2),
        day INT(2),
        hour INT(2),
        temp FLOAT(6,2),
        dewP FLOAT(4,2),
        humid FLOAT(5,2),
        windDir INT(3),
        windSpeed FLOAT(10,6),
        windGust FLOAT(10,6),
        precip FLOAT(2,2),
        pressure FLOAT(10,2),
        visib FLOAT(10,2),
        timeHour VARCHAR(25),
        PRIMARY KEY (weatherId)
        );
    '''
cursor.execute(sql)

Confirm the new tables were created. 

In [ ]:
sql = ' SHOW TABLES; '
cursor.execute(sql)
print ('The tables located in the myfltdata database are: \n')
for row in cursor:
    print (row)

#### Load the data into the newly created tables.

In [ ]:
# Load flights data into table

sql = '''
        LOAD DATA LOCAL INFILE 'flightsnew.csv'
        INTO TABLE flights
        FIELDS TERMINATED BY ','
        ENCLOSED BY '"'
        LINES TERMINATED BY '\r\n'
        IGNORE 1 ROWS;
    '''
cursor.execute(sql)
conn.commit()
print ("The number of rows added to the flight table: {}".format(cursor.rowcount))

In [ ]:
# Load weather data into table

sql = '''
        LOAD DATA LOCAL INFILE 'weathernew.csv'
        INTO TABLE weather
        FIELDS TERMINATED BY ','
        ENCLOSED BY '"'
        LINES TERMINATED BY '\r\n'
        IGNORE 1 ROWS;
    '''
cursor.execute(sql)
conn.commit()
print ("The number of rows added to the weather table: {}".format(cursor.rowcount))

#### Confirm data has been loaded into each table.

In [ ]:
# Confirm flight data

sql = ' SELECT * FROM flights WHERE flightId < 11;'
cursor.execute(sql)
for row in cursor.fetchall():
    print (row)

In [ ]:
# Confirm weather data

sql = ' SELECT * FROM weather WHERE weatherId < 11;'
cursor.execute(sql)
for row in cursor.fetchall():
    print (row)

### iii. Left join flights and weather on "year", "month", "day", "hour", and "origin"

#### Left Join Flights and Weather Tables

In [ ]:
sql = '''
    CREATE TABLE flights_weather AS (
    SELECT flights.flightId, flights.year, flights.month, flights.day, flights.hour, flights.depDelay, flights.arrDelay, flights.carrier, flights.origin, weather.temp, weather.humid, weather.windSpeed, weather.precip
    FROM flights
    LEFT JOIN weather ON flights.year = weather.year 
        AND flights.month = weather.month
        AND flights.day = weather.day
        AND flights.hour = weather.hour
        AND flights.origin = weather.origin
    );
    '''
cursor.execute(sql)
conn.commit()

Confirm the left join succeeded.

In [ ]:
sql = ' SHOW TABLES; '
cursor.execute(sql)
print ('The tables located in the myfltdata database are: \n')
for row in cursor:
    print (row)

In [ ]:
sql = ' SELECT * FROM flights_weather LIMIT 5'
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)

### iv. Select all flights that departed from JFK that were delayed in departure by more than 10 hours and store them in a separate table called mydelay

#### Select flights departing from JFK that were delayed > 10 hours

In [ ]:
sql = '''
    CREATE TABLE mydelay AS (
    SELECT * FROM flights_weather
    WHERE origin = "JFK"
    AND depDelay > 10)
    '''
cursor.execute(sql)
conn.commit()

Confirm the new table was created. 

In [ ]:
sql = ' SHOW TABLES; '
cursor.execute(sql)
print ('The tables located in the myfltdata database are: \n')
for row in cursor:
    print (row)

In [ ]:
sql = ' SELECT * FROM mydelay LIMIT 5'
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)

### v. What were the average weather conditions for these delayed flights at the time of scheduled departure in terms of temperature, humidity, wind speed and precipitation?


In [ ]:
mydelaypd = pd.read_sql('SELECT * FROM mydelay', conn)
mydelaypd.head()

In [ ]:
mdweather = mydelaypd[['temp', 'humid', 'windSpeed', 'precip']]
mdweather.describe()

In [ ]:
# Calculate the average weather conditions in for the delayed flights (pandas statistics is better)
sql = ' SELECT avg(temp), avg(humid), avg(windSpeed), avg(precip) FROM mydelay'
cursor.execute(sql)
for row in cursor.fetchall():
    print(row)

### vi. Provide plots of distributions for all these weather variables.

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [ ]:
# Remove any nan in the dataframe. 
mdweather.isnull().sum()
mdweather = mdweather[pd.notnull(mdweather['temp'])]

In [ ]:
ax = plt.axes()
sns.distplot(mdweather['temp'], ax=ax)
ax.set_title('Temperature')

In [ ]:
ax = plt.axes()
sns.distplot(mdweather['humid'], ax=ax)
ax.set_title('Humidity')

In [ ]:
ax = plt.axes()
sns.distplot(mdweather['windSpeed'], ax=ax)
ax.set_title('Wind Speed')

In [ ]:
ax = plt.axes()
sns.distplot(mdweather['precip'], ax=ax)
ax.set_title('Precipitation')

### vii.. Save data from mydelay into a csv file called mydelay.csv.

In [ ]:
mydelaypd.to_csv('mydelay.csv', index=False, quoting =1)

### viii. Read the mydelay.csv into a Pandas dataframe. In this file, is there a difference across airlines? how arrival delay is distributed? Please analyze this using histograms faceted by airlines.

In [ ]:
mydelaydf = pd.read_csv('mydelay.csv')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
airlinesdl = sns.FacetGrid(mydelaydf, row="year", col="carrier", margin_titles=True)
airlinesdl.map(plt.hist, "arrDelay", bins=np.linspace(0, 300, num=20));